### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [1]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    token = "", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\trini\Python\MTAA\Proyecto_Final\unsloth_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


C:\Users\trini\Python\MTAA\Proyecto_Final\unsloth_env\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.7.3: Fast Gemma3 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 3070 Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Versión CUDA compilada:", torch.version.cuda)
print("Dispositivo:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No detectado")


CUDA disponible: True
Versión CUDA compilada: 12.8
Dispositivo: NVIDIA GeForce RTX 3070 Laptop GPU


We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. 

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [5]:
import pandas as pd
train_set = pd.read_csv('./Datasets/train_set-ANT.csv', encoding="utf-8")
test_set = pd.read_csv('./Datasets/test_set-ANT.csv', encoding="utf-8")
val_set = pd.read_csv('./Datasets/val_set-ANT.csv', encoding="utf-8")

In [6]:
X_train = train_set['Neutralized SMILES']
y_train = train_set['ATC Codes']
X_test = test_set['Neutralized SMILES']
y_test = test_set['ATC Codes']
X_val = val_set['Neutralized SMILES']
y_val = val_set['ATC Codes']

In [7]:
for i in range(len(X_train)):
    smiles = X_train.at[i]
    s_smiles = smiles.replace("\\", "\\\\")
    X_train.at[i] = s_smiles
for i in range(len(X_test)):
    smiles = X_test.at[i]
    s_smiles = smiles.replace("\\", "\\\\")
    X_test.at[i] = s_smiles
for i in range(len(X_val)):
    smiles = X_val.at[i]
    s_smiles = smiles.replace("\\", "\\\\")
    X_val.at[i] = s_smiles

In [8]:
X_train

0       COc1cc2nc(N3CCN(C(=O)C4COc5ccccc5O4)CC3)nc(N)c...
1                 CC(C)(C)NC[C@@H](O)COc1cccc2c1CCC(=O)N2
2                             O=P(O)(O)C(Cl)(Cl)P(=O)(O)O
3                               Nc1cc(-c2ccncc2)c[nH]c1=O
4               CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21
                              ...                        
2434                              CC(C)c1ccccc1OCC1=NCCN1
2435       OC[C@H]1O[C@H](S[Au])[C@H](O)[C@@H](O)[C@@H]1O
2436    CN(C)c1cc(CNCC(C)(C)C)c(O)c2c1C[C@H]1C[C@H]3[C...
2437    Nc1nc(F)nc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)O)[C@...
2438        C[C@@H](NCCCc1cccc(C(F)(F)F)c1)c1cccc2ccccc12
Name: Neutralized SMILES, Length: 2439, dtype: object

In [ ]:
X_test

In [9]:
from datasets import Dataset
data  = []
for SMILES, ATCcode in zip(X_train, y_train):
    data.append({
        'conversations': [
            {'from': 'human', 'value': SMILES},
            {'from': 'gpt', 'value': ATCcode}
        ],
        'source': 'dataset_train',  
        'score': 1.0 
    })

dataset_train = Dataset.from_list(data)
print(dataset_train)

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 2439
})


In [10]:
data_test  = []
for SMILES, ATCcode in zip(X_test, y_test):
    data_test.append({
        'conversations': [
            {'from': 'human', 'value': SMILES},
            {'from': 'gpt', 'value': ATCcode}
        ],
        'source': 'dataset_test',  
        'score': 1.0 
    })

dataset_test = Dataset.from_list(data_test)
print(dataset_test)

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 2439
})


In [11]:
data_val  = []
for SMILES, ATCcode in zip(X_val, y_val):
    data_val.append({
        'conversations': [
            {'from': 'human', 'value': SMILES},
            {'from': 'gpt', 'value': ATCcode}
        ],
        'source': 'dataset_val',  
        'score': 1.0 
    })

dataset_val = Dataset.from_list(data_val)
print(dataset_val)

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 2439
})


In [12]:
dataset_train[4]

{'conversations': [{'from': 'human',
   'value': 'CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21'},
  {'from': 'gpt', 'value': 'J01MA; J01RA; S01AE'}],
 'source': 'dataset_train',
 'score': 1.0}

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [13]:
from unsloth.chat_templates import standardize_data_formats
dataset_train = standardize_data_formats(dataset_train)

Unsloth: Standardizing formats (num_proc=16): 100%|█████████████████████████| 2439/2439 [00:46<00:00, 52.03 examples/s]


In [14]:
dataset_test = standardize_data_formats(dataset_test)

Unsloth: Standardizing formats (num_proc=16): 100%|█████████████████████████| 2439/2439 [00:40<00:00, 60.70 examples/s]


In [15]:
dataset_val = standardize_data_formats(dataset_val)

Unsloth: Standardizing formats (num_proc=16): 100%|█████████████████████████| 2439/2439 [00:45<00:00, 53.35 examples/s]


In [16]:
dataset_train[4]

{'conversations': [{'content': 'CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21',
   'role': 'user'},
  {'content': 'J01MA; J01RA; S01AE', 'role': 'assistant'}],
 'source': 'dataset_train',
 'score': 1.0}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [17]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset_train = dataset_train.map(formatting_prompts_func, batched = True)
dataset_test = dataset_test.map(formatting_prompts_func, batched = True)

Map: 100%|███████████████████████████████████████████████████████████████| 2439/2439 [00:00<00:00, 15389.38 examples/s]


In [18]:
dataset_val = dataset_val.map(formatting_prompts_func, batched = True)

Map: 100%|███████████████████████████████████████████████████████████████| 2439/2439 [00:00<00:00, 15526.75 examples/s]


Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [19]:
dataset_train[100]["text"]

'<start_of_turn>user\nCC1(C)SC2C(NC(=O)C(C(=O)O)c3ccsc3)C(=O)N2C1C(=O)O<end_of_turn>\n<start_of_turn>model\nJ01CA<end_of_turn>\n'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [20]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [21]:
original_map = Dataset.map

def patched_map(self, *args, **kwargs):
    kwargs["num_proc"] = 1
    return original_map(self, *args, **kwargs)

Dataset.map = patched_map

In [22]:
for i, row in enumerate(dataset_train):
    try:
        tokenizer(row["text"])
    except Exception as e:
        print(f"Error en el ejemplo {i}: {e}")
        break

In [23]:
def formatting_func(example):
    return example["text"]

In [24]:
dataset_val[0]["text"]

'<start_of_turn>user\nCOc1cc2nc(N3CCN(C(=O)C4COc5ccccc5O4)CC3)nc(N)c2cc1OC<end_of_turn>\n<start_of_turn>model\nC02CA; G04CA<end_of_turn>\n'

In [25]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_val, # Can set up evaluation!
    formatting_func = formatting_func,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        #max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]: 100%|██████████████████████████████████████| 2439/2439 [00:00<00:00, 18684.11 examples/s]


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [26]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map: 100%|███████████████████████████████████████████████████████████████| 2439/2439 [00:00<00:00, 20870.12 examples/s]


Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [27]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nCC1(C)SC2C(NC(=O)C(C(=O)O)c3ccsc3)C(=O)N2C1C(=O)O<end_of_turn>\n<start_of_turn>model\nJ01CA<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [28]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                 J01CA<end_of_turn>\n'

In [29]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3070 Laptop GPU. Max memory = 8.0 GB.
4.385 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [30]:
import os
os.environ["TORCH_COMPILE_DISABLE"] = "1"
import torch._dynamo
torch._dynamo.config.disable = True

In [31]:
print(torch.__version__)
print(torch.version.cuda)
print(torch._dynamo.config.cache_size_limit)
print(torch._dynamo.config.repro_after)

2.7.1+cu128
12.8
1024
None


In [32]:
print(model.__class__)

<class 'peft.peft_model.PeftModelForCausalLM'>


In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,439 | Num Epochs = 3 | Total steps = 915
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,10.886400
2,10.043200
3,13.606400
4,7.911900
5,9.332600
6,7.811400
7,5.669100
8,5.207300
9,4.216600
10,3.710500


Unsloth: Will smartly offload gradients to save VRAM!


In [34]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3535.3628 seconds used for training.
58.92 minutes used for training.
Peak reserved memory = 8.41 GB.
Peak reserved memory for training = 4.025 GB.
Peak reserved memory % of max memory = 105.125 %.
Peak reserved memory for training % of max memory = 50.313 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [35]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : X_test[1], #N06BX
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, 
    tokenize=False # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nCC(C)(C)NC[C@@H](O)COc1cccc2c1CCC(=O)N2<end_of_turn>\n<start_of_turn>model\nC07AA<end_of_turn>']

In [36]:
print(X_test[1])
print(y_test[1])

CC(C)(C)NC[C@@H](O)COc1cccc2c1CCC(=O)N2
C07AA; S01ED


In [37]:
X_test

0       COc1cc2nc(N3CCN(C(=O)C4COc5ccccc5O4)CC3)nc(N)c...
1                 CC(C)(C)NC[C@@H](O)COc1cccc2c1CCC(=O)N2
2                             O=P(O)(O)C(Cl)(Cl)P(=O)(O)O
3                               Nc1cc(-c2ccncc2)c[nH]c1=O
4               CCn1cc(C(=O)O)c(=O)c2cc(F)c(N3CCNCC3)cc21
                              ...                        
2434                              CC(C)c1ccccc1OCC1=NCCN1
2435       OC[C@H]1O[C@H](S[Au])[C@H](O)[C@@H](O)[C@@H]1O
2436    CN(C)c1cc(CNCC(C)(C)C)c(O)c2c1C[C@H]1C[C@H]3[C...
2437    Nc1nc(F)nc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)O)[C@...
2438        C[C@@H](NCCCc1cccc(C(F)(F)F)c1)c1cccc2ccccc12
Name: Neutralized SMILES, Length: 2439, dtype: object

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [38]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : X_test[2230],}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize=False # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

A11HA<end_of_turn>


In [39]:
print(X_test[2230])
print(y_test[2230])

Cc1c(C)c2c(c(C)c1O)CC[C@@](C)(CCC[C@H](C)CCC[C@H](C)CCCC(C)C)O2
A11HA


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [40]:
model.save_pretrained("4gemma-3")  # Local saving
tokenizer.save_pretrained("4gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['4gemma-3\\processor_config.json']

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [41]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("4gemma-3-finetune", tokenizer)

Found HuggingFace hub cache directory: C:\Users\trini\.cache\huggingface\hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████████████████████████████████████████| 2/2 [19:09<00:00, 574.78s/it]


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [42]:
import os
print(os.environ["PATH"])

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Git\cmd;C:\ProgramData\ComposerSetup\bin;C:\Program Files\nodejs\;C:\Program Files\Docker\Docker\resources\bin;C:\Program Files\NVIDIA Corporation\Nsight Compute 2022.3.0\;C:\Program Files\NVIDIA Corporation\NVIDIA app\NvDLISR;C:\Users\trini\AppData\Local\Microsoft\WindowsApps;C:\Users\trini\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\trini\AppData\Local\GitHubDesktop\bin;C:\Users\trini\AppData\Roaming\Composer\vendor\bin;C:\Users\trini\php;C:\Users\trini\AppData\Roaming\npm;C:\

In [43]:
if True: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "4gemma-3-finetune",
        quantization_type = "q8_0", # For now only Q8_0, BF16, F16 supported
    )
    model.save_pretrained_gguf(
        "4gemma-3-finetune",
        quantization_type = "f16", # For now only Q8_0, BF16, F16 supported
    )

Unsloth GGUF:hf-to-gguf:Loading model: 4gemma-3-finetune
Unsloth GGUF:hf-to-gguf:Model architecture: Gemma3ForConditionalGeneration
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> Q8_0, shape = {2560, 262208}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:output_norm.weight,                torch.bfloat16 --> F32, shape = {2560}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:Set model quantization version
Unsloth GGUF:hf-to-gguf:Set model tokenizer
Unsloth GGUF:gguf.vocab:Setting special token type bos to 2
Unsloth GGUF:gguf.vocab:Setting special t

Unsloth: GGUF conversion: 100%|█████████████████████████████████████████| 100/100 [01:09<00:00,  1.69it/s, 4.12G/4.12G]

Unsloth GGUF:hf-to-gguf:Model successfully exported to .\


Unsloth: GGUF conversion: 100%|█████████████████████████████████████████| 100/100 [01:10<00:00,  1.43it/s, 4.12G/4.12G]

Unsloth: Converted to 4gemma-3-finetune.Q8_0.gguf with size = 4.1G


Unsloth: Successfully saved GGUF to:
4gemma-3-finetune.Q8_0.gguf
Unsloth GGUF:hf-to-gguf:Loading model: 4gemma-3-finetune
Unsloth GGUF:hf-to-gguf:Model architecture: Gemma3ForConditionalGeneration
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> F16, shape = {2560, 262208}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:output_norm.weight,                torch.bfloat16 --> F32, shape = {2560}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:Set model quantization version
Unsloth GGUF:hf-to-gguf:Set model tokenizer
Unsloth GGUF:gguf.vocab:Setting speci

Unsloth: GGUF conversion: 100%|█████████████████████████████████████████| 100/100 [00:40<00:00,  2.35it/s, 7.76G/7.76G]

Unsloth GGUF:hf-to-gguf:Model successfully exported to .\


Unsloth: GGUF conversion: 100%|█████████████████████████████████████████| 100/100 [00:40<00:00,  2.44it/s, 7.76G/7.76G]

Unsloth: Converted to 4gemma-3-finetune.F16.gguf with size = 7.8G
Unsloth: Successfully saved GGUF to:
4gemma-3-finetune.F16.gguf


In [ ]:
import nbformat

# Carga el notebook
path = "4-Gemma3(4B)finetuning.ipynb"
nb = nbformat.read(path, as_version=4)

# Elimina metadata.widgets pero mantiene el resto
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]

# Guarda el notebook corregido
nbformat.write(nb, path)

print("Metadata 'widgets' eliminada, outputs preservados.")
